In [1]:
import torch
import torch.nn as nn
import pandas as pd
import scipy.sparse as sp

In [3]:
# Define the model
class SparseMatrixFactorization(nn.Module):
    def __init__(self, n_users, n_movies, n_factors=20):
        super(SparseMatrixFactorization, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.movie_factors = nn.Embedding(n_movies, n_factors)
        
    def forward(self, user, movie):
        user_vector = self.user_factors(user)
        movie_vector = self.movie_factors(movie)
        dot_product = torch.sum(user_vector * movie_vector, dim=1)
        return dot_product

In [8]:
# Load the MovieLens dataset
movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

# Preprocess the data
n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId.unique().shape[0]

# Convert movieId and userId into unique integers
user_map = {u: i for i, u in enumerate(ratings.userId.unique())}
ratings['user_id'] = ratings['userId'].map(user_map)

movie_map = {m: i for i, m in enumerate(ratings.movieId.unique())}
ratings['movie_id'] = ratings['movieId'].map(movie_map)

# Create a sparse matrix with users as rows and movies as columns
matrix = sp.coo_matrix((ratings['rating'], (ratings['user_id'], ratings['movie_id'])))
matrix = matrix.tocsr()
print(matrix.row())

AttributeError: row not found

In [6]:
model = SparseMatrixFactorization(n_users, n_movies)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train the model
for i in range(100):
    optimizer.zero_grad()
    user = torch.LongTensor(matrix.row)
    movie = torch.LongTensor(matrix.col)
    rating = torch.FloatTensor(matrix.data)
    predictions = model(user, movie)
    loss = criterion(predictions, rating)
    loss.backward()
    optimizer.step()


AttributeError: row not found